In [1]:
import os
import torch
import numpy as np
import cv2 as cv
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import ae
import utils
from dataloader import BottleLoader
from ewm import ewm_G

In [3]:
# Set the root path of the AutoEncoder model experiments
exp_root = "/media/hdd1/kai/particle_generator/experiments/larcv_ae/"

In [4]:
# Set the root path where the ewm_code_vectors are stored
code_vec_root = "/media/hdd1/kai/particle_generator/ewm_code_vectors/"

## Get the names of all the saved AutoEncoder experiments in the exp_root folder

In [5]:
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

print("-"*60)
for i in range(len(exp_paths)):
    print("\n Exp_{}:".format(str(i)), exp_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_256 

------------------------------------------------------------

 Exp_1: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20_15000-epochs 

------------------------------------------------------------

 Exp_2: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20 

------------------------------------------------------------

 Exp_3: /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_128_20 

------------------------------------------------------------


## Select the AutoEncoder experiment you want

In [6]:
exp_dir = exp_paths[2]

In [7]:
# Create the full path to the AutoEncoder experiment
exp_path = os.path.join(exp_root, exp_dir) + "/"
print("Experiment path set as: \n{}".format(exp_path))

Experiment path set as: 
/media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/


In [8]:
# Path to model weights
weights_dir = "weights/"

In [9]:
# Set the GPU (GPU 1 is the best option)
device = torch.device(1)

## Load the AE config .csv as a dict
- Get the model architecture from the config_df

In [10]:
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [11]:
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].item())**2
im_dim   = int(np.sqrt(im_size))

In [12]:
# Set the code_vec_root to the appropriate path based on the model specs defined above
code_vec_path = code_vec_root + "ewm_code_vectors_{}_{}/".format(im_dim, l_dim)

## Set up AutoEncoder model on the GPU

In [13]:
# Set up AE layer sizes
base = [256] 

# Compute encoder sizes
sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
enc_sizes = base * n_layers
enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

# Update kwarg dicts
# Decoder is the reverse of the encoder
ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}

In [14]:
model = ae.AutoEncoder(**ae_kwargs).to(device)

## Load the AE model checkpoint

In [15]:
# Get checkpoint name(s)
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [16]:
print("-"*60)
for i in range(len(checkpoint_names)):
    print("\n Chkpt_{} :".format(str(i)), checkpoint_names[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Chkpt_0 : /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_7000.tar 

------------------------------------------------------------

 Chkpt_1 : /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_7499.tar 

------------------------------------------------------------

 Chkpt_2 : /media/hdd1/kai/particle_generator/experiments/larcv_ae/larcv_ae_64_20/weights/best_ae_ep_5250.tar 

------------------------------------------------------------


In [17]:
# Select the desired checkpoint from the list
best_checkpoint = checkpoint_names[1]

In [18]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(best_checkpoint)

In [19]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
model.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [26]:
# Put the model in evaluation mode
model.eval()

AutoEncoder(
  (encoder): Encoder(
    (fc_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=4096, out_features=2048, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=2048, out_features=1024, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (2): Sequential(
        (0): Linear(in_features=1024, out_features=512, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (3): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (last): Linear(in_features=256, out_features=20, bias=True)
  )
  (decoder): Decoder(
    (fc_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=20, out_features=256, bias=True)
        (1): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=512, bias=True)
        (1): Leaky

## Set the directory where the model outputs should be saved

In [ ]:
# TODO: Write this section after the checking that model deploys properly

In [ ]:
path = "/media/hdd1/kai/particle_generator/ewm_code_vectors/ewm_code_vectors_64_20/ewm_code_vectors_64_20/"
paths = []
for ex in os.listdir(path):
    print(os.path.join(path, ex))

In [ ]:
path = code_vec_path.strip('/').split('_')
dims = []
for item in path:
    try:
        dims.append(int(item))
    except ValueError:
        pass
dims

## Deploy the Decoder using the set of ewm generated code_vectors

### Setup the BottleLoader

In [21]:
loader_kwargs = {'num_workers' : 1, 'batch_size': 1, 'shuffle': True}
train_transform = transforms.Compose([ transforms.ToTensor() ]) 
test_dataset = BottleLoader(root = code_vec_path, transforms=train_transform)

Code-Target examples will be loaded from subfolder of: 
/media/hdd1/kai/particle_generator/ewm_code_vectors/ewm_code_vectors_64_20/


In [22]:
dataloader = DataLoader(test_dataset, **loader_kwargs)

## Run ewm_code_vectors through the Decoder

In [50]:
samples = []; limit = 24
for idx, code_vec in enumerate(dataloader):
    
    # Move code_vector onto GPU
    code_vec = code_vec.to(device)
    
    # Get sample from model
    sample = model.decoder(code_vec)
    
    # Reshape the sample, detach, and convert to numpy array
    samples.append(sample.view(im_dim, im_dim).detach().cpu().numpy())
    
    if idx == limit:
        break

In [51]:
print(samples[0].shape, type(samples[0]))

(64, 64) <class 'numpy.ndarray'>


## Viz the deploy samples

In [52]:
n_row = 3; n_col = limit // n_row

fig, axes = plt.subplots(n_row, n_col)
supTitle = "{} model deploy samples using {}x{} dataset \n and {}_dim code vectors".format('Decoder', im_dim, im_dim, l_dim)
fig.suptitle(supTitle, color="white")

img = None; test_num = limit; sample_num = limit
for i in range(0, n_row):
    for j in range(0, n_col):
        img = axes[i,j].imshow( samples[sample_num] )
        axes[i,j].spines['bottom'].set_color('white')
        axes[i,j].spines['top'].set_color('white')
        axes[i,j].spines['left'].set_color('white')
        axes[i,j].spines['right'].set_color('white')
        axes[i,j].xaxis.label.set_color('white')
        axes[i,j].yaxis.label.set_color('white')
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        axes[i,j].tick_params(axis='x', colors='#443941')
        axes[i,j].tick_params(axis='y', colors='#443941')
        sample_num -= 1

# Colorbar
# cbar_axes = fig.add_axes([0.9, 0.1, 0.03, 0.8])
# cbar = plt.colorbar(img, cax=cbar_axes)
# cbar.ax.yaxis.set_tick_params(color="white")
# cbar.outline.set_edgecolor(color="white")
# plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color="white")

plt.savefig("{}_model_deploy_samples_{}_dataset_{}_code_vectors.png".format('Decoder', im_dim, l_dim), dpi=300, facecolor='#443941')